In [1]:
# imports
import numpy as np
import pandas as pd
import pickle
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from tqdm.auto import tqdm
from multiprocessing import Pool

In [2]:
# connect to local elastic search host
HOST = 'http://localhost:9200/'
es = Elasticsearch(hosts=[HOST])
INDEX = 'info_clone'
NEWINDEX = 'info_clone2'
es.ping()

True

In [3]:
t = es.indices.stats(index=INDEX)
SIZE = t['_all']['primaries']['docs']['count']

In [4]:
s = Search(using=es, index=INDEX)
q = Q("wildcard", name='*')
query = s.query(q) #.extra(size=SIZE)
#response = query.execute()
#print(len(response))

In [16]:
# Find files that contain no markdown or code
def check_empty(es, nid):
    mkdn = es.get(index='markdown', id=nid, ignore=[404])
    mkdnlist = mkdn['_source']['markdown']
    code = es.get(index='code', id=nid, ignore=[404])
    codelist = code['_source']['code']
    comments = es.get(index='comments2', id=nid, ignore=[404])
    cmtlist = comments['_source']['comments']
    if mkdnlist == [] and codelist == [] and cmtlist == []:
        return True
    else:
        return False

In [ ]:
# POOL NOT WORKING
# SLICES = 5                                                                      

# def dump_slice(slice_no):
#     empty_files = []
#     s = Search()                                                                
#     s = s.extra(slice={"id": slice_no, "max": SLICES})                          
#     for hit in s.scan():                                                          
#         nid = hit.meta.id
#         if check_empty(es, nid, name):
#             empty_files.append(nid)
#     return empty_files

# pool = Pool(SLICES)                                                             
# pool.map(dump_slice, range(SLICES))

empty_files = []
for hit in s.scan():
    nid = hit.meta.id
    if check_empty(es, nid):
        print(nid, end='\r')
        empty_files.append(nid)

In [13]:
len(empty_files)

0

In [ ]:
with open('empty.pkl', 'wb') as f:
    pickle.dump(empty_files, f)

In [ ]:
with open('empty.pkl', 'rb') as f:
     empty_files = pickle.load(f)

In [ ]:
#es.delete(index=NEWINDEX, id=nid)
#mdelete

In [ ]:
# Does not run properly
SLICES = 2

def test(slice_no):
    test = []
    for i in range(10):
        test.append(i)
    return test

pool = Pool(SLICES)                                                             
all_test = pool.map(test, range(SLICES))
all_test = [i for l in all_test for i in l]